# A common interface for handling tabular data

As we've seen in the FITS tutorial, the [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html) sub-package can be used to access FITS tables. In addition, as we will see in the next tutorial, there is functionality in [astropy.io.votable](http://docs.astropy.org/en/stable/io/votable/index.html) and [astropy.io.ascii](http://docs.astropy.org/en/stable/io/ascii/index.html) to read in VO and ASCII tables. However, while these sub-pacakges have user interfaces that are specific to each kind of file, it can be difficult to remember all of them. Therefore, astropy includes a higher level interface in [astropy.table](http://docs.astropy.org/en/stable/table/index.html) which can be used to access tables in many different formats in a similar way.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create tables</li>
<li>Access data in tables</li>
<li>Combining tables</li>
<li>Using high-level objects as columns</li>
<li>Aggregation</li>
<li>Masking</li>
<li>Reading/writing</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy.table. For more information about the features presented below as well as other available features, you can read the
[astropy.table documentation](https://docs.astropy.org/en/stable/table/).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Creating tables

The main class we will use here is called ``Table``:

In [2]:
from astropy.table import Table

Before we look at how to read and write tables, let's first see how to create a table from scratch:

In [3]:
t1 = Table()
t1['name'] = ['source1', 'source2', 'source3']
t1['flux'] = [1.1, 1.2, 1.3]

We can look at the table with:

In [4]:
t1

name,flux
str7,float64
source1,1.1
source2,1.2
source3,1.3


We can add columns:

In [6]:
t1['size'] = [1, 5, 4]
t1

name,flux,size
str7,float64,int64
source1,1.1,1
source2,1.2,5
source3,1.3,4


Access the values in a column:

In [7]:
t1['size']

1
5
4


In [8]:
t1['flux'][1]

1.2

Convert the column to a Numpy array:

In [9]:
import numpy as np
np.array(t1['size'])

array([1, 5, 4])

Access individual cells:

In [10]:
t1['size'][0]

1

And access rows:

In [11]:
t1[0]

name,flux,size
str7,float64,int64
source1,1.1,1


fun quantity things:

In [13]:
t1['size']>1

array([False,  True,  True])

## Units in tables

Table columns can include units:

In [14]:
from astropy import units as u
t1['size'].unit = u.cm
t1['flux'].unit = 'mJy'
t1

name,flux,size
,mJy,cm
str7,float64,int64
source1,1.1,1
source2,1.2,5
source3,1.3,4


Some unitful operations will then work:

In [19]:
t1['size'].to('m')

<Quantity [0.01, 0.05, 0.04] m>

In [20]:
type(t1['size'].to('m'))

astropy.units.quantity.Quantity

In [21]:
type(t1['size'])

astropy.table.column.Column

However, you may run into unexpected behavior, so if you are planning on using table columns as Quantities, we recommend that you use the ``QTable`` class:

In [22]:
t1

name,flux,size
,mJy,cm
str7,float64,int64
source1,1.1,1
source2,1.2,5
source3,1.3,4


In [23]:
from astropy.table import QTable
qt1 = QTable(t1)

In [24]:
qt1

name,flux,size
,mJy,cm
str7,float64,float64
source1,1.1,1.0
source2,1.2,5.0
source3,1.3,4.0


In [25]:
type(qt1['size'])

astropy.units.quantity.Quantity


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<ol>
<li>Make a table that contains three columns: <code>spectral type</code>, <code>temperature</code>, and <code>radius</code>, and incude 5 rows with fake data (or real data if you like, for example from <a href="http://www.atlasoftheuniverse.com/startype.html">here</a>). Try including units on the columns that can have them.</li>
<li>Find the mean temperature and the maximum radius</li>
<li>Try and find out how to add and remove rows</li>
<li>Add a new column which gives the luminosity (using $L=4\pi R^2 \sigma T^4$)</li>
</ol>

</div>

</section>


## Iterating over tables

It is possible to iterate over rows or over columns. To iterate over rows, iterate over the table itself:

In [26]:
for row in t1:
    print(row)

  name  flux size
        mJy   cm 
------- ---- ----
source1  1.1    1
  name  flux size
        mJy   cm 
------- ---- ----
source2  1.2    5
  name  flux size
        mJy   cm 
------- ---- ----
source3  1.3    4


Rows can act like dictionaries, so you can access specific columns from a row:

In [28]:
for row in t1:
    print(row['name'])

source1
source2
source3


Iterating over columns is also easy:

In [31]:
for colname in t1.columns:
    column = t1[colname]
    print(column)

  name 
-------
source1
source2
source3
flux
mJy 
----
 1.1
 1.2
 1.3
size
 cm 
----
   1
   5
   4


Accessing specific rows from a column object can be done with the item notation:

In [33]:
for colname in t1.columns:
    column = t1[colname]
    print(column[0])

source1
1.1
1


You can do this with QTables, too. QTables have extra functionality:

In [34]:
for row in qt1:
    print(row['size'], type(row['size']))

1.0 cm <class 'astropy.units.quantity.Quantity'>
5.0 cm <class 'astropy.units.quantity.Quantity'>
4.0 cm <class 'astropy.units.quantity.Quantity'>


## Joining tables

The astropy.table sub-package provides a few useful functions for stacking/combining tables. For example, we can do a 'join':

add: https://docs.astropy.org/en/stable/table/operations.html#table-operations

In [37]:
t2 = Table()
t2['name'] = ['source1', 'source3']
t2['flux2'] = [1, 9]
t2

name,flux2
str7,int64
source1,1
source3,9


There are lots of ways to do this- vstack, hstack, cstack (different ways to put tables together)

In [38]:
from astropy.table import join

In [39]:
t3 = join(t1, t2, join_type='outer')
t3

name,flux,size,flux2
,mJy,cm,
str7,float64,int64,int64
source1,1.1,1,1
source2,1.2,5,--
source3,1.3,4,9


In [40]:
np.mean(t3['flux2'])

5.0

## Masked tables

It is possible to mask individual cells in tables:

In [41]:
t4 = Table(masked=True)

In [42]:
t4['id'] = [4, 5, 6]
t4['flux'] = [1.3, 1.5, 1.6]

In [43]:
t4

id,flux
int64,float64
4,1.3
5,1.5
6,1.6


In [44]:
t4['flux'].mask = [1, 0, 1]
t4

id,flux
int64,float64
4,--
5,1.5
6,--


In [45]:
select = np.array([False, True, True])

In [47]:
t5 = t4[select]
t5

id,flux
int64,float64
5,1.5
6,--


## Using high-level objects as columns

A few specific astropy high-level objects can be used as columns in table - this includes SkyCoord and Time:

In [48]:
from astropy.time import Time
from astropy.coordinates import SkyCoord

In [49]:
t6 = Table()

In [50]:
t6['time'] = Time([50000, 51000, 52000], format='mjd')

In [51]:
t6['coord'] = SkyCoord([1, 2, 3] * u.deg, [4, 5, 6] * u.deg)

In [52]:
t6['flux'] = [1, 5, 4] * u.mJy

In [53]:
t6

time,coord,flux
,"deg,deg",mJy
object,object,float64
50000.0,"1.0,4.0",1.0
51000.0,"2.0,5.0",5.0
52000.0,"3.0,6.0",4.0


In [54]:
t6[0]['coord']

<SkyCoord (ICRS): (ra, dec) in deg
    (1., 4.)>

Note however that you may not necessarily be able to write this table to a file and get it back intact, since being able to store this kind of information is not possible in all file formats.

## Slicing

Tables can be sliced like Numpy arrays:

In [55]:
obs = Table(rows=[('M31' , '2012-01-02', 17.0, 17.5),
                  ('M31' , '2012-01-02', 17.1, 17.4),
                  ('M101', '2012-01-02', 15.1, 13.5),
                  ('M82' , '2012-02-14', 16.2, 14.5),
                  ('M31' , '2012-02-14', 16.9, 17.3),
                  ('M82' , '2012-02-14', 15.2, 15.5),
                  ('M101', '2012-02-14', 15.0, 13.6),
                  ('M82' , '2012-03-26', 15.7, 16.5),
                  ('M101', '2012-03-26', 15.1, 13.5),
                  ('M101', '2012-03-26', 14.8, 14.3)],
            names=['name', 'obs_date', 'mag_b', 'mag_v'])

In [56]:
obs

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M31,2012-01-02,17.0,17.5
M31,2012-01-02,17.1,17.4
M101,2012-01-02,15.1,13.5
M82,2012-02-14,16.2,14.5
M31,2012-02-14,16.9,17.3
M82,2012-02-14,15.2,15.5
M101,2012-02-14,15.0,13.6
M82,2012-03-26,15.7,16.5
M101,2012-03-26,15.1,13.5


In [57]:
obs[1:4]

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M31,2012-01-02,17.1,17.4
M101,2012-01-02,15.1,13.5
M82,2012-02-14,16.2,14.5


In [58]:
obs[obs['mag_b'] > 15.5]

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M31,2012-01-02,17.0,17.5
M31,2012-01-02,17.1,17.4
M82,2012-02-14,16.2,14.5
M31,2012-02-14,16.9,17.3
M82,2012-03-26,15.7,16.5


In [59]:
obs['mag_b', 'mag_v']

mag_b,mag_v
float64,float64
17.0,17.5
17.1,17.4
15.1,13.5
16.2,14.5
16.9,17.3
15.2,15.5
15.0,13.6
15.7,16.5
15.1,13.5



<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Starting from the <code>obs</code> table:</p>
<ol>
<li>Make a new table that shows every other row, starting with the second row? (that is, the second, fourth, sixth, etc. rows).</li>
<li>Make a new table the only contains rows where <code>name</code> is <code>M31</code></li>
</ol>

</div>

</section>


## Grouping and Aggregation

It is possible to aggregate rows of a table together - for example, to group the rows by source name in the ``obs`` table, you can do:

In [60]:
obs

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M31,2012-01-02,17.0,17.5
M31,2012-01-02,17.1,17.4
M101,2012-01-02,15.1,13.5
M82,2012-02-14,16.2,14.5
M31,2012-02-14,16.9,17.3
M82,2012-02-14,15.2,15.5
M101,2012-02-14,15.0,13.6
M82,2012-03-26,15.7,16.5
M101,2012-03-26,15.1,13.5


In [61]:
obs_by_name = obs.group_by('name')

In [62]:
obs_by_name

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M101,2012-01-02,15.1,13.5
M101,2012-02-14,15.0,13.6
M101,2012-03-26,15.1,13.5
M101,2012-03-26,14.8,14.3
M31,2012-01-02,17.0,17.5
M31,2012-01-02,17.1,17.4
M31,2012-02-14,16.9,17.3
M82,2012-02-14,16.2,14.5
M82,2012-02-14,15.2,15.5


This is not just sorting the values but actually making it possible to access each group of rows:

In [63]:
for group in obs_by_name.groups:
    print(group)
    print("")

name  obs_date  mag_b mag_v
---- ---------- ----- -----
M101 2012-01-02  15.1  13.5
M101 2012-02-14  15.0  13.6
M101 2012-03-26  15.1  13.5
M101 2012-03-26  14.8  14.3

name  obs_date  mag_b mag_v
---- ---------- ----- -----
 M31 2012-01-02  17.0  17.5
 M31 2012-01-02  17.1  17.4
 M31 2012-02-14  16.9  17.3

name  obs_date  mag_b mag_v
---- ---------- ----- -----
 M82 2012-02-14  16.2  14.5
 M82 2012-02-14  15.2  15.5
 M82 2012-03-26  15.7  16.5



In [64]:
obs_by_name.groups

<TableGroups indices=[ 0  4  7 10]>

We can then aggregate the rows together in each group using a function:

In [65]:
obs_by_name.groups.aggregate(np.mean)

name,mag_b,mag_v
str4,float64,float64
M101,15.000000000000002,13.725000000000001
M31,17.0,17.400000000000002
M82,15.699999999999998,15.5


In [66]:
obs_by_name['name', 'mag_v'].groups.aggregate(np.mean)

name,mag_v
str4,float64
M101,13.725000000000001
M31,17.400000000000002
M82,15.5


## Writing data

To write out the data, we can use the ``write`` method:

In [67]:
obs

name,obs_date,mag_b,mag_v
str4,str10,float64,float64
M31,2012-01-02,17.0,17.5
M31,2012-01-02,17.1,17.4
M101,2012-01-02,15.1,13.5
M82,2012-02-14,16.2,14.5
M31,2012-02-14,16.9,17.3
M82,2012-02-14,15.2,15.5
M101,2012-02-14,15.0,13.6
M82,2012-03-26,15.7,16.5
M101,2012-03-26,15.1,13.5


In [68]:
obs.write('test.fits')

In [69]:
obs.write('test.tex')

In some cases the format will be inferred from the extension, but only in unambiguous cases - otherwise the format has to be specified explicitly:

In [70]:
obs.write('test.vot', format='votable', overwrite=True)

You can find the [list of supported formats](https://docs.astropy.org/en/stable/io/unified.html#built-in-table-readers-writers) in the documentation.

## Reading data

You can also easily read in tables using the ``read`` method:

In [71]:
t7 = Table.read('data/2mass.tbl', format='ascii.ipac')

In [72]:
Table.read.help('ascii')

Table.read(format='ascii') documentation

Read the input ``table`` and return the table.  Most of
the default behavior for various parameters is determined by the Reader
class.

See also:

- https://docs.astropy.org/en/stable/io/ascii/
- https://docs.astropy.org/en/stable/io/ascii/read.html

Parameters
----------
table : str, file-like, list, `pathlib.Path` object
    Input table as a file name, file-like object, list of string[s],
    single newline-separated string or `pathlib.Path` object.
guess : bool
    Try to guess the table format. Defaults to None.
format : str, `~astropy.io.ascii.BaseReader`
    Input table format
Inputter : `~astropy.io.ascii.BaseInputter`
    Inputter class
Outputter : `~astropy.io.ascii.BaseOutputter`
    Outputter class
delimiter : str
    Column delimiter string
comment : str
    Regular expression defining a comment line in table
quotechar : str
    One-character string to quote fields containing special characters
header_start : int
    Line index for 

In [74]:
t8 = Table.read('data/gaia_lmc_psc.fits')

In [75]:
t8

source_id,ra,ra_error,dec,dec_error,parallax,parallax_error,phot_g_mean_mag,bp_rp,radial_velocity,radial_velocity_error,phot_variable_flag,teff_val,a_g_val
int64,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,bytes13,float32,float32
4650802592000604416,87.07819921385541,0.021177289402850533,-71.9758462572808,0.023118971922399856,1.9515300334170036,0.022129316045590156,9.447254,1.3508034,60.90398378334771,0.2792654428569048,NOT_AVAILABLE,4558.4004,0.292
4654524816824470144,74.41054299130985,0.023043629903234147,-71.69279844885818,0.023510209030041487,0.9532188638663136,0.02339411705460298,10.067117,1.5253868,-14.48511977709958,0.22425351318898115,NOT_AVAILABLE,4297.3867,0.2825
4654529695907256832,74.20814124067418,0.07882566958428994,-71.61632605005579,0.05787066182603759,2.3901953811128607,0.05452097913656001,9.930226,1.3181801,46.38448485466361,2.490830888596897,NOT_AVAILABLE,4637.565,nan
4654557218058933760,73.18224206495864,0.02566576394382618,-71.57203874651691,0.023184091706528888,8.467976171104247,0.024934154792585914,9.095835,0.6556463,49.49827252135888,0.2219686958314872,NOT_AVAILABLE,6255.75,0.132
5279853466498770816,94.44753106184642,0.03057782849457327,-68.60869574722126,0.04155980825556449,4.915939636800765,0.03590801116219821,10.310742,0.74726105,37.385157965047355,0.4835527467303836,NOT_AVAILABLE,5915.3003,0.309
4662917595229314944,73.09902402736357,0.031716989446635716,-66.24090336494024,0.02434918668808062,1.2843391476325927,0.027022816638180115,10.124996,1.1030893,27.543957838023847,1.320355612047364,NOT_AVAILABLE,4978.6665,nan
4662931304765244288,72.79394552845352,0.07339225726421537,-65.97530694819791,0.07784922165620814,4.073463462498102,0.07694028124440017,10.244692,0.7755623,nan,nan,NOT_AVAILABLE,5813.5,nan
4662942334232773888,72.43114765425396,1.687482616173613,-65.96502066188053,1.4133043528817208,nan,nan,10.448099,1.0799913,nan,nan,NOT_AVAILABLE,5095.6333,nan
4662942329938049024,72.43067172359882,0.6134998277695469,-65.96503044762075,0.6321611818144194,nan,nan,10.3262005,1.0531683,nan,nan,NOT_AVAILABLE,5095.6333,nan


In [77]:
t9 = Table.read('data/xmm-log.vot')

In [78]:
t9

Obsno,PropDate,Object,RAJ2000,DEJ2000,Obs0,ObsDur,Image,PPSp,FITS,XSAlink,SASVersion
,s,,deg,deg,s,s,,,,,
str10,str10,str30,float64,float64,str19,int32,str1,str4,str4,str1,str5
0000110101,2002-09-29,XTE J0421+560,64.92542,55.99944,2001-08-19T07:05:23,32913,Y,PPSp,FITS,Y,9.0
0001730101,2004-12-31,,--,--,2002-03-18T06:40:01,25296,N,PPSp,FITS,N,NOPPS
0001730201,2002-05-25,HD159176,263.67495,-32.58167,2001-03-09T12:44:21,17083,Y,PPSp,FITS,Y,9.0
0001730301,2002-05-25,HD159176,263.67495,-32.58167,2001-03-09T17:30:16,9362,N,PPSp,FITS,Y,9.0
0001730401,2002-05-25,HD159176,263.67495,-32.58167,2001-03-09T09:41:25,10859,N,PPSp,FITS,Y,9.0
0001730501,2004-12-31,HD47129,99.34999,6.13528,2002-09-17T18:35:28,21939,N,PPSp,FITS,Y,9.0
0001730601,2004-12-31,HD47129,99.34999,6.13528,2003-03-16T16:01:51,21863,Y,PPSp,FITS,Y,9.0
0001930101,2002-09-18,IRAS F00235+1024,6.52917,10.68917,2001-01-10T18:47:04,26609,Y,PPSp,FITS,Y,9.0



<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Using the <code>t7</code> (2MASS) table above:</p>
<ol>
<li>
<p>Make a plot that shows <code>j_m</code>-<code>h_m</code> on the x-axis, and <code>h_m</code>-<code>k_m</code> on the y-axis</p>
</li>
<li>
<p>Make a new table that contains the subset of rows where the <code>j_snr</code>, <code>h_snr</code>, and <code>k_snr</code> columns, which give the signal-to-noise-ratio in the J, H, and K band, are greater than 10, and try and show these points in red in the plot you just made.</p>
</li>
<li>
<p>Make a new table (based on the full table) that contains only the RA, Dec, and the <code>j_m</code>, <code>h_m</code> and <code>k_m</code> columns, then try and write out this catalog into a format that you can read into another software package. For example, try and write out the catalog into CSV format, then read it into a spreadsheet software package (e.g. Excel, Google Docs, Numbers, OpenOffice). You may run into an issue at this point - if so, take a look at https://github.com/astropy/astropy/issues/7357 to see how to fix it.</p>
</li>
</ol>

</div>

</section>


<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)